<a href="https://colab.research.google.com/github/suadamohammed/noise_classifier/blob/main/Vision_Transformer_image__classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vit-keras

!pip install tensorflow-addons
import pandas as pd

  Preparing metadata (setup.py) ... done
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=44e4a72eba48095a9ec82c86797759f6d19a5c7fbeddf79154e76d5947767ece
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 41.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/phD/noise_classification/dataset

/content/drive/.shortcut-targets-by-id/14YS8pA9eHMVUAZUd982tmtu9mpuO2_oD/phD/noise_classification/dataset


In [12]:
import os
import numpy as np
from PIL import Image
from PIL import Image, UnidentifiedImageError
base_path = './train_im/'
X = []
Y = []
classCount = 0

source_path = base_path
for child in os.listdir(source_path):
    classCount += 1
    sub_path = os.path.join(source_path, child)
    if os.path.isdir(sub_path):
        for data_file in os.listdir(sub_path)[:]:
            try:
                if os.path.join(sub_path, data_file):
                    Qry = Image.open(os.path.join(sub_path, data_file))
                    Qry = np.array(Qry.resize((32, 32)))

                    feature = Qry
                    X.append(feature)
                    Y.append(child)
            except (UnidentifiedImageError, IsADirectoryError):
                print("failed",sub_path)
                continue

print('No of classes:', classCount)

failed ./train_im/speckle
No of classes: 9


In [13]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models, layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

"""encoder = LabelEncoder()
y = encoder.fit_transform(Y)
print(y)  """
"""y = pd.get_dummies(y)
print(y)"""
x_train, x_test, y_train, y_test = train_test_split(np.array(X), np.array(Y),
                                                    test_size=0.3, random_state=42)

In [14]:
x_train_rgb = tf.repeat(x_train[..., tf.newaxis], 3, axis=-1)
x_test_rgb = tf.repeat(x_test[..., tf.newaxis], 3, axis=-1)

In [15]:

#not for gray images,only for RGB

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications.vgg16 import preprocess_input
from vit_keras import vit
x_train = preprocess_input(x_train_rgb)
x_test = preprocess_input(x_test_rgb)

In [16]:

from sklearn.preprocessing import LabelEncoder

# Encode the string labels into integer values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert the encoded labels to categorical format
y_train_categorical = tf.keras.utils.to_categorical(y_train_encoded, num_classes=9)
y_test_categorical = tf.keras.utils.to_categorical(y_test_encoded, num_classes=9)


In [17]:
import tensorflow as tf

image_size = 32

# Resize images using tf.image.resize
x_train_resized = tf.image.resize(x_train, (image_size, image_size))


In [ ]:

x_test_resized = tf.image.resize(x_test, (image_size, image_size))

# Build ViT model
model = vit.vit_b16(
    image_size=image_size,
    activation="softmax",
    pretrained=True,
    include_top=True,
    pretrained_top=False,
    classes=9,
)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"],
)

# Train the model
batch_size = 32
epochs = 100
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=1)
model.fit(x_train_resized[:], y_train_categorical[:], batch_size=batch_size, epochs=epochs)

Epoch 1/100
291/291 [==============================] - 60s 56ms/step - loss: 2.0018 - accuracy: 0.2344
Epoch 2/100
291/291 [==============================] - 16s 55ms/step - loss: 1.8341 - accuracy: 0.2713
Epoch 3/100
291/291 [==============================] - 16s 56ms/step - loss: 1.7858 - accuracy: 0.2814
Epoch 4/100
291/291 [==============================] - 17s 57ms/step - loss: 1.7198 - accuracy: 0.3091
Epoch 5/100
291/291 [==============================] - 16s 57ms/step - loss: 1.6801 - accuracy: 0.3217
Epoch 6/100
291/291 [==============================] - 16s 55ms/step - loss: 1.6370 - accuracy: 0.3401
Epoch 7/100
291/291 [==============================] - 16s 55ms/step - loss: 1.6044 - accuracy: 0.3467
Epoch 8/100
291/291 [==============================] - 16s 55ms/step - loss: 1.5707 - accuracy: 0.3619
Epoch 9/100
291/291 [==============================] - 16s 55ms/step - loss: 1.5529 - accuracy: 0.3688
Epoch 10/100
291/291 [==============================] - 16s 56ms/step - l

In [21]:
import numpy as np
import pandas as pd

# Assuming your matrix is stored in a variable called X and class labels in Y
num_samples, height, width = X.shape

# Flatten each image into a 1D array
flattened_images = X.reshape(num_samples, height * width)

# Convert the flattened images and class labels to a DataFrame
df = pd.DataFrame(flattened_images)
df['class_label'] = Y

# Save the DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/phD/noise_classification/featureslabel_128__allimages.csv', index=False)


In [ ]:
model.save('/content/drive/MyDrive/phD/noise_classification/vit_model.h5')

In [ ]:



testloss, testAccuracy=model.evaluate(x_test_resized,y_test_categorical)

print(testAccuracy)



In [ ]:
y_pred = (model.predict(x_test_resized) > 0.5).astype("int32")

In [ ]:
#print(y_pred)

In [ ]:
# print(y_test_one_hot)
# print(np.argmax(y_pred, axis=1))

In [ ]:
#print(type(y_pred))

#print(type(y_test.values))
#print(y_pred)
#print(y_test.values)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
import seaborn as sns

#np.argmax(y_test.values, axis=1)
print(classification_report(y_test_categorical,y_pred))

cm = confusion_matrix(np.argmax(y_test_categorical, axis=1),np.argmax(y_pred, axis=1))

sns.heatmap(cm,cmap='Greens', annot=True, xticklabels = ['erlang', 'exponential', 'gaussian', 'lognormal', 'poisson', 'rayleigh', 'saltpepper', 'speckle','uniform'],yticklabels = ['erlang', 'exponential', 'gaussian', 'lognormal', 'poisson', 'rayleigh', 'saltpepper', 'speckle','uniform'])


In [ ]:
# Train a classifier
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
X_test_flattened = X_test.reshape(X_test.shape[0], -1)

clf = RandomForestClassifier()
ovr = OneVsRestClassifier(clf)
ovr.fit(X_train_flattened, y_train_categorical)

# Predict the class probabilities for the test data
y_scores = ovr.predict_proba(X_test_flattened)

# Compute the ROC curve and AUC for each class
y_test1 = np.argmax(y_test_categorical, axis=1)
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(y_scores.shape[1]):
    fpr[i], tpr[i], _ = roc_curve(y_test1 == i, y_scores[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute the macro-average ROC curve and AUC
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(y_scores.shape[1])]))
mean_tpr = np.zeros_like(all_fpr)
for i in range(y_scores.shape[1]):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
mean_tpr /= y_scores.shape[1]
macro_auc = auc(all_fpr, mean_tpr)

class_labels = ["Kathi", "Kari", "Pacha", "Minuk", "Thadi"]

# Plot the ROC curve for each class and the macro-average ROC curve
plt.figure()
plt.plot([0, 1], [0, 1], 'k--')

for i in range(y_scores.shape[1]):
    plt.plot(fpr[i], tpr[i], label='{} (AUC = {:.2f})'.format(class_labels[i], roc_auc[i]))
plt.plot(all_fpr, mean_tpr, label='Macro-average (AUC = {:.2f})'.format(macro_auc), lw=2, color='navy')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass ROC Curve')
plt.legend(loc="lower right")
plt.show()



In [26]:
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from tensorflow.python.ops.array_ops import shape
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# Train a classifier
#X_train = x_train_resized.reshape(x_train_resized.shape[0], -1)
#X_test = x_test_resized.reshape(x_test_resized.shape[0], -1)
X_train = x_train_resized
X_test = x_test_resized
y_test_one_hot=y_test_categorical
y_train_one_hot=y_train_categorical
clf = RandomForestClassifier()
ovr = OneVsRestClassifier(clf)


ovr.fit(X_train, y_train_one_hot)




# Predict the class probabilities for the test data
y_scores = ovr.predict_proba(X_test)

print(y_scores)
#print(y_test.values)

# Compute the ROC curve and AUC score for each class

# roc curve for classes

thresh ={}

n_class = 5
y_test1= np.argmax(y_test_one_hot, axis=1)
print(y_test1)
# Compute the ROC curve and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()


for i in range(y_scores.shape[1]):
    fpr[i], tpr[i], _ = roc_curve(y_test1 == i, y_scores[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute the macro-average ROC curve and AUC
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(y_scores.shape[1])]))
mean_tpr = np.zeros_like(all_fpr)
for i in range(y_scores.shape[1]):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
mean_tpr /= y_scores.shape[1]
macro_auc = auc(all_fpr, mean_tpr)

class_labels = ["Kathi", "Kari", "Pacha", "Minuk", "Thadi"]

# Plot the ROC curve for each class and the macro-average ROC curve
plt.figure()
plt.plot([0, 1], [0, 1], 'k--')


for i in range(y_scores.shape[1]):
    plt.plot(fpr[i], tpr[i], label='{} (AUC = {:.2f})'.format(class_labels[i], roc_auc[i]))
plt.plot(all_fpr, mean_tpr, label='Macro-average (AUC = {:.2f})'.format(macro_auc), lw=2, color='navy')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass ROC Curve')
plt.legend(loc="lower right")
plt.show()

ValueError: ignored